## Extracting region of interest
##### the code used to load the trained model was adapted from https://github.com/matterport/Mask_RCNN/blob/master/samples/balloon/inspect_balloon_data.ipynb


In [ ]:
#importing the necessary modules
import os
import sys
import tensorflow as tf
from imutils import paths
import cv2
import numpy
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR) 
from samples.balloon import balloon
import mrcnn.model as modellib

## Load the trained model

In [ ]:
# define the directory of the model and the model weights
model_directory_path = "PATH/TO/Mask_RCNN-master/logs/PROJECT_NAME/"
weights_path = "PATH/TO/Mask_RCNN-master/logs/PROJECT_NAME/WEIGHTS_NAME.h5"

#use the same configuration as the balloon example
config = balloon.BalloonConfig()
DEVICE = "/gpu:0"  #define if images are going to be processed in the gpu or the cpu

#open the model
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=model_directory_path,
                              config=config)
#load the weights
model.load_weights(weights_path, by_name=True)

## Extract the region of interest

In [ ]:
#directory of the pictures to be extracted
Original_data_set="PATH/TO/ORIGINAL_PICTURES"
#list all pictures from the directory
imagePaths = sorted(list(paths.list_images(Original_data_set)))

#loop through all pictures in the directory and extract the region of interest
for c in range(0, len(imagePaths)):
    #load the picture
    image = cv2.imread(imagePaths[c])
    # Run object detector
    results = model.detect([image], verbose=0)[0]
    #loop through the "results" to extract all regions of interest in the image
    for i in range(0,results["masks"].shape[2]):
        #get the coordinates of the box containing the region of interest
        x=results["rois"][i][0]
        y=results["rois"][i][1]
        w=results["rois"][i][2]
        h=results["rois"][i][3]
        
        #transform every pixel that is not part of the region of interest to black
        mask=results["masks"][:, :, i].astype(numpy.uint8)
        contours, im2 = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        stencil = numpy.zeros(image.shape).astype(image.dtype)
        color = [255, 255, 255]
        cv2.fillPoly(stencil, contours, color)
        result = cv2.bitwise_and(image, stencil)
        
        #extract the box containing the region of interest and save the image
        result=result[x:w,y:h]
        cv2.imwrite("PATH/TO/EXTRACTED_PICTURES_FOLDER/"+str(i)+"_"+imagePaths[c].split("/")[-1:][0], result)